# Milestone 5: Agentic RAG with Multi-Tool Orchestration

This notebook demonstrates agentic RAG using multiple tools:
1. Retriever tool for document search
2. Citation checker for verification
3. Summarizer for answer refinement
4. Agent orchestration with LangGraph

## Setup

In [ ]:
# Import required modules
import sys
sys.path.append('..')

from src import agent_rag

print("Agentic RAG module loaded successfully!")

## Step 1: Understand Individual Tools

Explore each tool independently.

In [ ]:
# Create individual tools
retriever = agent_rag.RetrieverTool()
citation_checker = agent_rag.CitationCheckerTool()
summarizer = agent_rag.SummarizerTool()

print("Tools created:")
print(f"1. {retriever.name}: {retriever.description}")
print(f"2. {citation_checker.name}: {citation_checker.description}")
print(f"3. {summarizer.name}: {summarizer.description}")

## Step 2: Test Retriever Tool

Test document retrieval independently.

In [ ]:
# Test retriever
query = "What are the principles of data processing?"

print(f"Query: {query}")
print("\nRetrieving documents...")

retrieved = retriever.run(query)

print("\nRetrieved:")
print("=" * 60)
print(retrieved)
print("=" * 60)

## Step 3: Test Citation Checker

Verify citations in an answer.

In [ ]:
# Test citation checker
answer_with_sources = """
Answer: GDPR requires data to be processed lawfully and fairly.

Sources:
Article 5: Principles relating to processing of personal data
Article 6: Lawfulness of processing
"""

print("Checking citations...\n")
verification = citation_checker.run(answer_with_sources)

print("Verification Result:")
print("=" * 60)
print(verification)
print("=" * 60)

## Step 4: Test Summarizer

Summarize a long response.

In [ ]:
# Test summarizer
long_text = """The General Data Protection Regulation (GDPR) is a comprehensive 
data protection law that was adopted by the European Union. It came into effect 
on May 25, 2018, replacing the 1995 Data Protection Directive. The regulation 
aims to harmonize data protection laws across the EU and give individuals greater 
control over their personal data. It applies to all organizations processing 
personal data of EU residents, regardless of where the organization is located. 
The regulation introduces several key principles including lawfulness, fairness, 
transparency, purpose limitation, data minimization, accuracy, storage limitation, 
integrity, and confidentiality."""

print("Original text length:", len(long_text), "characters\n")
print("Summarizing...\n")

summary = summarizer.run(long_text)

print("Summary:")
print("=" * 60)
print(summary)
print("=" * 60)
print("\nSummary length:", len(summary), "characters")

## Step 5: Agent Orchestration

Use AgentRunner to orchestrate all tools.

In [ ]:
# Create agent runner
runner = agent_rag.AgentRunner()

print("Agent Runner initialized")
print(f"Available tools: {list(runner.tools.keys())}")

## Step 6: Run Complete Agentic Workflow

Execute the full workflow with all tools.

In [ ]:
# Run complete workflow
query = "What rights do data subjects have under GDPR?"

print(f"Query: {query}")
print("\n" + "=" * 60)

result = runner.run_workflow(query)

print("\n" + "=" * 60)
print("\nFINAL RESULT:")
print(f"Query: {result['query']}")
print(f"\nAnswer: {result['answer']}")
print(f"\nVerification: {result['verification']}")
print(f"\nSteps executed: {', '.join(result['steps'])}")

## Step 7: Test Multiple Queries

Run the agent on different types of questions.

In [ ]:
# Test queries
test_queries = [
    "What is the right to erasure?",
    "Explain data portability",
    "What are the penalties for non-compliance?"
]

print("Testing Agentic RAG on multiple queries:")
print("=" * 60)

for i, q in enumerate(test_queries, 1):
    print(f"\n{i}. {q}")
    result = runner.run_workflow(q)
    print(f"   Answer: {result['answer'][:120]}...")
    print(f"   Verification: {result['verification']}")

## Step 8: LangGraph Agent Creation

Create a LangGraph-based agent (skeleton).

In [ ]:
# Create LangGraph agent
tools = [retriever, citation_checker, summarizer]

langgraph_agent = agent_rag.create_langgraph_agent(tools)

print("\nLangGraph Agent:")
print(f"  Status: {langgraph_agent}")
print(f"  Tools: {[t.name for t in tools]}")

print("\nNote: Full LangGraph implementation requires:")
print("  - from langgraph.prebuilt import create_react_agent")
print("  - LLM instance")
print("  - Tool definitions in LangGraph format")

## Step 9: Workflow Comparison

Compare simple RAG vs. agentic RAG.

In [ ]:
# Comparison
from src import rag_baseline

simple_rag = rag_baseline.BaselineRAG()
query = "What is consent in GDPR?"

print("Comparison: Simple RAG vs. Agentic RAG")
print("=" * 60)

print("\n1. SIMPLE RAG:")
simple_result = simple_rag.query(query)
print(f"   Answer: {simple_result['answer'][:150]}...")
print(f"   Sources: {simple_result['num_sources']}")

print("\n2. AGENTIC RAG:")
agentic_result = runner.run_workflow(query)
print(f"   Answer: {agentic_result['answer'][:150]}...")
print(f"   Verification: {agentic_result['verification']}")
print(f"   Steps: {len(agentic_result['steps'])}")

print("\nAdvantages of Agentic RAG:")
print("  - Multiple tool orchestration")
print("  - Citation verification")
print("  - Answer refinement")
print("  - More controllable workflow")

## Summary

In this notebook, we:
- ✅ Created and tested individual tools (Retriever, Citation Checker, Summarizer)
- ✅ Implemented agent orchestration
- ✅ Ran complete agentic workflows
- ✅ Tested with multiple queries
- ✅ Explored LangGraph integration
- ✅ Compared simple vs. agentic RAG

Next: Proceed to `06_graph_rag.ipynb` for graph-enhanced retrieval.